# Eat Safe, Love

## Notebook Set Up

In [2]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
print(mongo.list_database_names())

['admin', 'autosaurus', 'class_db', 'config', 'epa', 'fruits_db', 'local', 'met', 'mydatabase', 'petsitly_marketing', 'uk_food']


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
show = {'BusinessName': 1, 'scores.Hygiene': 1}
result = establishments.find(query, show)

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of documents are: " + str(count))

# Display the first document in the results using pprint
pprint(result[0])

Number of documents are: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('66eb3ac2d2f882ed7ac62d34'),
 'scores': {'Hygiene': 20}}


In [8]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
count_rows = len(hygiene_df["BusinessName"])
print("Number of rows are: " + str(count_rows))

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

Number of rows are: 41


,_id,BusinessName,scores
0,66eb3ac2d2f882ed7ac62d34,The Chase Rest Home,{'Hygiene': 20}
1,66eb3ac2d2f882ed7ac630b7,Brenalwood,{'Hygiene': 20}
2,66eb3ac2d2f882ed7ac633c2,Melrose Hotel,{'Hygiene': 20}
3,66eb3ac2d2f882ed7ac635ae,Seaford Pizza,{'Hygiene': 20}
4,66eb3ac2d2f882ed7ac635bf,Golden Palace,{'Hygiene': 20}
5,66eb3ac2d2f882ed7ac63f5d,Ashby's Butchers,{'Hygiene': 20}
6,66eb3ac2d2f882ed7ac6417e,South Sea Express Cuisine,{'Hygiene': 20}
7,66eb3ac2d2f882ed7ac656a6,Golden Palace,{'Hygiene': 20}
8,66eb3ac2d2f882ed7ac65aeb,The Tulip Tree,{'Hygiene': 20}
9,66eb3ac2d2f882ed7ac662fd,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [9]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName':{'$regex':'London'},'RatingValue': {'$gte': 4}}
show = {'BusinessName': 1, 'LocalAuthorityName': 1, 'RatingValue': 1}
result = establishments.find(query, show)


# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of documents are: " + str(count))


# Display the first document in the results using pprint
pprint(result[0])

Number of documents are: 33
{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('66eb3ac2d2f882ed7ac64753')}


In [10]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
count_rows = len(london_df["BusinessName"])

# Display the first 10 rows of the DataFrame
london_df.head(20)

,_id,BusinessName,RatingValue,LocalAuthorityName
0,66eb3ac2d2f882ed7ac64753,Charlie's,4,City of London Corporation
1,66eb3ac2d2f882ed7ac64a78,Mv City Cruises Erasmus,5,City of London Corporation
2,66eb3ac2d2f882ed7ac655c4,Benfleet Motor Yacht Club,4,City of London Corporation
3,66eb3ac2d2f882ed7ac663c6,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,66eb3ac2d2f882ed7ac663cb,Tilbury Seafarers Centre,5,City of London Corporation
5,66eb3ac2d2f882ed7ac66ac9,Mv Valulla,5,City of London Corporation
6,66eb3ac2d2f882ed7ac68bdc,Tereza Joanne,5,City of London Corporation
7,66eb3ac2d2f882ed7ac68fa3,Brick Lane Brews,4,City of London Corporation
8,66eb3ac2d2f882ed7ac68fb8,The Nuance Group (UK) Limited,5,City of London Corporation
9,66eb3ac2d2f882ed7ac68fba,City Bar & Grill,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [13]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Find Penang Flavours
query_1 = {'BusinessName': 'Penang Flavours'}

show_1 = {'BusinessName': 1,'geocode.latitude': 1, 'geocode.longitude': 1}
result_1 = establishments.find(query_1, show_1)

degree_search = 0.01
latitude = result_1[0]['geocode']['latitude']
longitude = result_1[0]['geocode']['longitude']

# Top five
query = {'RatingValue': 5,
         'geocode.latitude': {'$gte': (latitude - degree_search)},
         'geocode.latitude': {'$lte': (latitude + degree_search)},
         'geocode.longitude': {'$gte': (longitude - degree_search)},
         'geocode.longitude': {'$lte': (longitude + degree_search)},
         'scores.Hygiene': {'$ne': None}
         }

show = {'BusinessName': 1,'RatingValue': 1, 'scores.Hygiene':1, 'geocode.latitude': 1, 'geocode.longitude': 1}
result = establishments.find(query, show).sort('scores.Hygiene', 1).limit(5)


# Print the results
BusinessName_list =[]
RatingValue_list = []
latitude_list = []
longitude_list = []
Hygiene_list = []

for document in result:
    pprint(document)
    BusinessName = document['BusinessName']
    Rating = document['RatingValue']
    latitude = document['geocode']['latitude']
    longitude = document['geocode']['longitude']
    Hygiene = document['scores']['Hygiene']
    BusinessName_list.append(BusinessName)
    RatingValue_list.append(Rating)
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    Hygiene_list.append(Hygiene)

{'BusinessName': 'Clifden House Dementia Care Centre',
 'RatingValue': 5,
 '_id': ObjectId('66eb3ac2d2f882ed7ac63764'),
 'geocode': {'latitude': 50.776112, 'longitude': 0.089886},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Beachlands Care Home',
 'RatingValue': 5,
 '_id': ObjectId('66eb3ac2d2f882ed7ac63766'),
 'geocode': {'latitude': 50.775392, 'longitude': 0.088903},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'The Tide at The Buckle',
 'RatingValue': 5,
 '_id': ObjectId('66eb3ac2d2f882ed7ac637fc'),
 'geocode': {'latitude': 50.779367, 'longitude': 0.08091},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Costcutter Claremont',
 'RatingValue': 5,
 '_id': ObjectId('66eb3ac2d2f882ed7ac63712'),
 'geocode': {'latitude': 50.77601, 'longitude': 0.093059},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Angies Newsagents',
 'RatingValue': 5,
 '_id': ObjectId('66eb3ac2d2f882ed7ac63713'),
 'geocode': {'latitude': 50.77601, 'longitude': 0.093059},
 'scores': {'Hygiene': 0}}


In [14]:
# Convert result to Pandas DataFrame
rate_df = pd.DataFrame({'BusinessName': BusinessName_list,
                          'RatingValue': RatingValue_list,
                          'latitude': latitude_list,
                          'longitude': longitude_list,
                          'Hygiene': Hygiene_list })

rate_df.head()

,BusinessName,RatingValue,latitude,longitude,Hygiene
0,Clifden House Dementia Care Centre,5,50.776112,0.089886,0
1,Beachlands Care Home,5,50.775392,0.088903,0
2,The Tide at The Buckle,5,50.779367,0.080910,0
3,Costcutter Claremont,5,50.776010,0.093059,0
4,Angies Newsagents,5,50.776010,0.093059,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [15]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results
pipeline = [{'$match': {'LocalAuthorityName': {'$ne': None},'scores.Hygiene': 0}},
            {'$group': {'_id': '$LocalAuthorityName','count': {'$sum': 1}}},
            {'$sort': {'count': -1}},
            {'$limit': 10}]

result = establishments.aggregate(pipeline)


LocalAuthorityName_list = []
count_list =[]

for document in result:
    print(document)
    name = document['_id']
    LocalAuthorityName_list.append(name)
    number = document['count']
    count_list.append(number)

{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [16]:
# Convert the result to a Pandas DataFrame
hygiene1_df = pd.DataFrame({"LocalAuthorityName": LocalAuthorityName_list,"count":count_list})

# Display the number of rows in the DataFrame
count_rows = len(hygiene1_df["LocalAuthorityName"])
print("Number of rows are: " + str(count_rows))

# Display the first 10 rows of the DataFrame
hygiene1_df.head(10)

Number of rows are: 10


,LocalAuthorityName,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
